In [1]:
import gym
import numpy as np
import torch
import time

from gym.wrappers import Monitor

import torch
from torch.distributions import Categorical
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import copy
import os

from itertools import count

import matplotlib.pyplot as plt
%matplotlib notebook

# NEURO-EVO

In [2]:
class CartPoleAgent(nn.Module):
    def __init__(self):
        super(CartPoleAgent, self).__init__()
        self.affine1 = nn.Linear(4, 124)
        # self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(124, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        # x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

In [3]:
def get_initialized_agents(folderName):
    agents = []
    for path in os.listdir(folderName):
        if path[-4:] == '.pth':
            try:
                model = CartPoleAgent()
                model.load_state_dict(torch.load(folderName + '/' + path))
                agents.append(model)
            except Exception as e:
                print(e)
    return agents

    get_initialized_agents(folderName)

In [4]:
def run_agents(agents):
    game_actions = 2
    reward_agents = []
    env = gym.make("CartPole-v1")
    env.spec.reward_threshold = 500
    
    for agent in agents:
        agent.eval()
    
        observation = env.reset()
        
        r, s = 0, 0
        for _ in range(250):
            
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env.step(action)
            r = r + reward
            
            s = s + 1
            observation = new_observation

            if done:
                break

        reward_agents.append(r)        
        # reward_agents.append(s)
    
    return reward_agents

In [5]:
def return_average_score(agent, runs):
    score = 0.
    for i in range(runs):
        score += run_agents([agent])[0]
    return score / runs

In [6]:
def run_agents_n_times(agents, runs):
    return [return_average_score(agent, runs) for agent in agents]

In [7]:
def mutate(agent):
    child_agent = copy.deepcopy(agent)
    mutation_power = 0.02 # Set from https://arxiv.org/pdf/1712.06567.pdf
    for param in child_agent.parameters():
        if len(param.shape) == 4: # Weights of Conv2D
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):
                            param[i0][i1][i2][i3] += mutation_power * np.random.randn()
        
        elif len(param.shape) == 2: # Weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    param[i0][i1] += mutation_power * np.random.randn()
        
        elif len(param.shape) == 1: # Biases of linear layer or conv layer
            for i0 in range(param.shape[0]):
                
                param[i0] += mutation_power * np.random.randn()

    return child_agent

In [8]:
def return_children(agents, sorted_parent_indexes, elite_index):
    children_agents = []
    
    for i in range(len(agents)-1):
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index = len(children_agents) - 1
    
    return children_agents, elite_index

In [9]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    
    if elite_index is not None:
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])
        
    top_score = None
    top_elite_index = None
    
    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=5)
        print("Score for elite i ", i, " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [10]:
def play_agent(agent):
    try:
        env = gym.make("CartPole-v1")
        
        env_record = Monitor(env, './video', force=True)
        observation = env_record.reset()
        last_observation = observation
        
        r = 0
        for _ in range(250):
            env_record.render()
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env_record.step(action)
            r=r+reward
            observation = new_observation

            if done:
                break

        env_record.close()
        print("Rewards: ", r)

    except Exception as e:
        env_record.close()
        print(e.__doc__)
        print(e.message)        

In [11]:
def trainEVOModel(folderName, generationsRun, returnScore = False):
    game_actions = 2
    torch.set_grad_enabled(False)

    agents = get_initialized_agents('./' + folderName)

    top_limit = 5 # Number of top agents to consider as parents
    generations = generationsRun

    elite_index = None
    for generation in range(generations):
        rewards = run_agents_n_times(agents, 10) # Average of k runs

        sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit]
        print('\n')

        top_rewards = []
        for best_parent in sorted_parent_indexes:
            top_rewards.append(rewards[best_parent])

        print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
        # print(rewards)
        print("Top ",top_limit," scores", sorted_parent_indexes)
        print("Rewards for top: ",top_rewards)

        children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)
        agents = children_agents
        
    if returnScore == True:
        return np.mean(reward)

# DQN

In [12]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.state_space = env.observation_space.shape[0]
        self.action_space = env.action_space.n
        self.affine1 = nn.Linear(self.state_space, 124)
        #self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(124, self.action_space)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        #x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

In [13]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()

def sim_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    actions = policy(state)
    _, action = actions.max(1)
    return action.item()

def finish_episode():
    R = 0
    policy_loss = []
    returns = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

def sim():
    tot_reward = 0
    state = env.reset()
    for t in range(1, 10000):
        action = sim_action(state)
        state, reward, done, _ = env.step(action)
        tot_reward += reward
        env.render()
        if done:
            print(tot_reward)
            break

In [14]:
def trainDQNmodel(returnReward = False):
    running_reward = 10
    for i_episode in range(numEpisodes):
        state, ep_reward = env.reset(), 0
        for t in range(1, 20000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if render:
                env.render()
            policy.rewards.append(reward)
            ep_reward += reward
            if done:
                #duration.append(t)
                break

        running_reward = 0.04 * ep_reward + (1 - 0.04) * running_reward
        finish_episode()
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        #if running_reward > env.spec.reward_threshold:
         #   print("Solved! Running reward is now {} and "
        #          "the last episode runs to {} time steps!".format(running_reward, t))
         #   print("{},{}".format(i_episode, ep_reward))
         #   env.close()
         #   break
    if returnReward == True:
        env.close()
        return running_reward
    else:
        return
        

In [15]:
gamma = 0.99 # discount factor
seed  = 543
render = False
log_interval = 10

env = gym.make('CartPole-v1')
#numEpisodes = 15
#numSuccessions = 10

In [16]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1.10e-2)
eps = np.finfo(np.float32).eps.item()

# STOP RUNNING HERE

In [ ]:
for successions in range(numSuccessions):
    
    folderName = "Succession" + str(successions)
    if not(os.path.isdir(folderName)):
        os.mkdir(folderName)
    
    for episodes in range(numEpisodes):
        policy = Policy()
        trainDQNmodel()
        fileName   = "Episode" + str(episodes) + ".pth"
        torch.save(policy.state_dict(), "./" + folderName + "/" + fileName)
        
    trainEVOModel(folderName, fileName)

In [ ]:
env.spec.reward_threshold

In [ ]:
from timeit import default_timer as timer

numEpisodesList = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
numSuccessions = 100

avgReward = np.zeros(len(numEpisodesList))
avgTimeToRun = np.zeros(len(numEpisodesList))

for ii in range(len(numEpisodesList)):
    
    reward    = 0
    timeToRun = 0

    numEpisodes = numEpisodesList[ii]

    for successions in range(numSuccessions):
        
        print(numEpisodesList[ii])
        print(successions)
        start   = timer()
        
        policy = Policy()
        optimizer = optim.Adam(policy.parameters(), lr=1.10e-2)
        eps = np.finfo(np.float32).eps.item()
        #env = gym.make('CartPole-v1')
        reward += trainDQNmodel(returnReward = True)
        end     = timer()
        
        timeToRun = (end - start)
            
    avgReward[ii]    = reward/numSuccessions
    avgTimeToRun[ii] = timeToRun/numSuccessions

50
0
Episode 0	Last reward: 23.00	Average reward: 10.52
Episode 10	Last reward: 17.00	Average reward: 23.60
Episode 20	Last reward: 57.00	Average reward: 34.65
Episode 30	Last reward: 21.00	Average reward: 34.18
Episode 40	Last reward: 53.00	Average reward: 34.26
50
1
Episode 0	Last reward: 18.00	Average reward: 10.32
Episode 10	Last reward: 9.00	Average reward: 11.64
Episode 20	Last reward: 25.00	Average reward: 13.46
Episode 30	Last reward: 50.00	Average reward: 21.46
Episode 40	Last reward: 57.00	Average reward: 39.04
50
2
Episode 0	Last reward: 16.00	Average reward: 10.24
Episode 10	Last reward: 92.00	Average reward: 34.92
Episode 20	Last reward: 402.00	Average reward: 70.85
Episode 30	Last reward: 260.00	Average reward: 111.40
Episode 40	Last reward: 138.00	Average reward: 129.26
50
3
Episode 0	Last reward: 25.00	Average reward: 10.60
Episode 10	Last reward: 16.00	Average reward: 12.81
Episode 20	Last reward: 41.00	Average reward: 17.56
Episode 30	Last reward: 38.00	Average reward

50
31
Episode 0	Last reward: 59.00	Average reward: 11.96
Episode 10	Last reward: 10.00	Average reward: 11.95
Episode 20	Last reward: 9.00	Average reward: 11.66
Episode 30	Last reward: 9.00	Average reward: 10.95
Episode 40	Last reward: 9.00	Average reward: 10.88
50
32
Episode 0	Last reward: 15.00	Average reward: 10.20
Episode 10	Last reward: 37.00	Average reward: 14.83
Episode 20	Last reward: 14.00	Average reward: 16.02
Episode 30	Last reward: 20.00	Average reward: 17.10
Episode 40	Last reward: 130.00	Average reward: 38.54
50
33
Episode 0	Last reward: 14.00	Average reward: 10.16
Episode 10	Last reward: 75.00	Average reward: 19.22
Episode 20	Last reward: 93.00	Average reward: 49.81
Episode 30	Last reward: 22.00	Average reward: 65.35
Episode 40	Last reward: 25.00	Average reward: 54.77
50
34
Episode 0	Last reward: 28.00	Average reward: 10.72
Episode 10	Last reward: 40.00	Average reward: 15.75
Episode 20	Last reward: 26.00	Average reward: 22.68
Episode 30	Last reward: 53.00	Average reward: 

50
62
Episode 0	Last reward: 22.00	Average reward: 10.48
Episode 10	Last reward: 23.00	Average reward: 13.59
Episode 20	Last reward: 22.00	Average reward: 21.96
Episode 30	Last reward: 31.00	Average reward: 30.54
Episode 40	Last reward: 22.00	Average reward: 34.48
50
63
Episode 0	Last reward: 33.00	Average reward: 10.92
Episode 10	Last reward: 16.00	Average reward: 16.07
Episode 20	Last reward: 27.00	Average reward: 23.86
Episode 30	Last reward: 40.00	Average reward: 34.92
Episode 40	Last reward: 57.00	Average reward: 55.57
50
64
Episode 0	Last reward: 53.00	Average reward: 11.72
Episode 10	Last reward: 10.00	Average reward: 11.17
Episode 20	Last reward: 10.00	Average reward: 10.59
Episode 30	Last reward: 11.00	Average reward: 10.21
Episode 40	Last reward: 10.00	Average reward: 9.99
50
65
Episode 0	Last reward: 34.00	Average reward: 10.96
Episode 10	Last reward: 35.00	Average reward: 18.07
Episode 20	Last reward: 76.00	Average reward: 37.88
Episode 30	Last reward: 239.00	Average reward

Episode 10	Last reward: 12.00	Average reward: 13.62
Episode 20	Last reward: 56.00	Average reward: 23.62
Episode 30	Last reward: 27.00	Average reward: 31.30
Episode 40	Last reward: 44.00	Average reward: 32.24
50
94
Episode 0	Last reward: 12.00	Average reward: 10.08
Episode 10	Last reward: 10.00	Average reward: 11.37
Episode 20	Last reward: 14.00	Average reward: 14.01
Episode 30	Last reward: 35.00	Average reward: 20.26
Episode 40	Last reward: 71.00	Average reward: 30.97
50
95
Episode 0	Last reward: 12.00	Average reward: 10.08
Episode 10	Last reward: 11.00	Average reward: 10.69
Episode 20	Last reward: 11.00	Average reward: 10.50
Episode 30	Last reward: 10.00	Average reward: 10.22
Episode 40	Last reward: 8.00	Average reward: 9.88
50
96
Episode 0	Last reward: 52.00	Average reward: 11.68
Episode 10	Last reward: 15.00	Average reward: 16.25
Episode 20	Last reward: 86.00	Average reward: 25.16
Episode 30	Last reward: 76.00	Average reward: 36.32
Episode 40	Last reward: 16.00	Average reward: 46.89

Episode 20	Last reward: 15.00	Average reward: 12.53
Episode 30	Last reward: 16.00	Average reward: 13.49
Episode 40	Last reward: 33.00	Average reward: 19.03
Episode 50	Last reward: 74.00	Average reward: 32.36
Episode 60	Last reward: 30.00	Average reward: 36.59
Episode 70	Last reward: 34.00	Average reward: 34.41
Episode 80	Last reward: 59.00	Average reward: 36.22
Episode 90	Last reward: 37.00	Average reward: 41.98
100
13
Episode 0	Last reward: 32.00	Average reward: 10.88
Episode 10	Last reward: 10.00	Average reward: 10.78
Episode 20	Last reward: 10.00	Average reward: 10.40
Episode 30	Last reward: 10.00	Average reward: 9.97
Episode 40	Last reward: 9.00	Average reward: 9.81
Episode 50	Last reward: 10.00	Average reward: 9.75
Episode 60	Last reward: 9.00	Average reward: 9.50
Episode 70	Last reward: 10.00	Average reward: 9.41
Episode 80	Last reward: 11.00	Average reward: 9.51
Episode 90	Last reward: 9.00	Average reward: 9.44
100
14
Episode 0	Last reward: 14.00	Average reward: 10.16
Episode 10

...

Episode 160	Last reward: 77.00	Average reward: 68.27
Episode 170	Last reward: 102.00	Average reward: 68.38
Episode 180	Last reward: 60.00	Average reward: 66.23
Episode 190	Last reward: 43.00	Average reward: 59.22
Episode 200	Last reward: 51.00	Average reward: 53.14
Episode 210	Last reward: 44.00	Average reward: 47.89
Episode 220	Last reward: 37.00	Average reward: 44.66
Episode 230	Last reward: 84.00	Average reward: 48.75
Episode 240	Last reward: 93.00	Average reward: 52.45
Episode 250	Last reward: 57.00	Average reward: 57.11
Episode 260	Last reward: 27.00	Average reward: 55.69
Episode 270	Last reward: 29.00	Average reward: 47.34
Episode 280	Last reward: 23.00	Average reward: 39.73
Episode 290	Last reward: 25.00	Average reward: 35.78
Episode 300	Last reward: 33.00	Average reward: 32.79
Episode 310	Last reward: 36.00	Average reward: 31.80
Episode 320	Last reward: 24.00	Average reward: 32.71
Episode 330	Last reward: 41.00	Average reward: 33.49
Episode 340	Last reward: 37.00	Average reward

Episode 300	Last reward: 21.00	Average reward: 113.71
Episode 310	Last reward: 186.00	Average reward: 121.31
Episode 320	Last reward: 500.00	Average reward: 217.93
Episode 330	Last reward: 500.00	Average reward: 312.47
Episode 340	Last reward: 342.00	Average reward: 348.87
350
83
Episode 0	Last reward: 13.00	Average reward: 10.12
Episode 10	Last reward: 18.00	Average reward: 14.44
Episode 20	Last reward: 37.00	Average reward: 22.23
Episode 30	Last reward: 97.00	Average reward: 32.35
Episode 40	Last reward: 129.00	Average reward: 57.47
Episode 50	Last reward: 83.00	Average reward: 66.36
Episode 60	Last reward: 133.00	Average reward: 74.41
Episode 70	Last reward: 116.00	Average reward: 105.06
Episode 80	Last reward: 95.00	Average reward: 92.75
Episode 90	Last reward: 105.00	Average reward: 89.37
Episode 100	Last reward: 77.00	Average reward: 90.42
Episode 110	Last reward: 155.00	Average reward: 105.26
Episode 120	Last reward: 156.00	Average reward: 115.00
Episode 130	Last reward: 177.00	

Episode 80	Last reward: 10.00	Average reward: 10.05
Episode 90	Last reward: 9.00	Average reward: 9.91
Episode 100	Last reward: 9.00	Average reward: 9.84
Episode 110	Last reward: 11.00	Average reward: 9.94
Episode 120	Last reward: 12.00	Average reward: 9.97
Episode 130	Last reward: 8.00	Average reward: 9.74
Episode 140	Last reward: 10.00	Average reward: 9.69
Episode 150	Last reward: 9.00	Average reward: 9.60
Episode 160	Last reward: 10.00	Average reward: 9.49
Episode 170	Last reward: 10.00	Average reward: 9.49
Episode 180	Last reward: 9.00	Average reward: 9.62
Episode 190	Last reward: 9.00	Average reward: 9.55
Episode 200	Last reward: 9.00	Average reward: 9.44
Episode 210	Last reward: 10.00	Average reward: 9.52
Episode 220	Last reward: 9.00	Average reward: 9.57
Episode 230	Last reward: 9.00	Average reward: 9.61
Episode 240	Last reward: 11.00	Average reward: 9.66
Episode 250	Last reward: 12.00	Average reward: 9.76
Episode 260	Last reward: 10.00	Average reward: 9.64
Episode 270	Last rewar

Episode 220	Last reward: 394.00	Average reward: 437.64
Episode 230	Last reward: 363.00	Average reward: 425.14
Episode 240	Last reward: 500.00	Average reward: 437.17
Episode 250	Last reward: 500.00	Average reward: 452.72
Episode 260	Last reward: 134.00	Average reward: 427.06
Episode 270	Last reward: 111.00	Average reward: 321.95
Episode 280	Last reward: 114.00	Average reward: 250.32
Episode 290	Last reward: 142.00	Average reward: 206.09
Episode 300	Last reward: 233.00	Average reward: 209.61
Episode 310	Last reward: 122.00	Average reward: 189.62
Episode 320	Last reward: 113.00	Average reward: 165.26
Episode 330	Last reward: 128.00	Average reward: 149.66
Episode 340	Last reward: 124.00	Average reward: 140.42
350
92
Episode 0	Last reward: 16.00	Average reward: 10.24
Episode 10	Last reward: 14.00	Average reward: 10.89
Episode 20	Last reward: 13.00	Average reward: 11.64
Episode 30	Last reward: 40.00	Average reward: 19.01
Episode 40	Last reward: 92.00	Average reward: 32.41
Episode 50	Last rew

Episode 10	Last reward: 14.00	Average reward: 15.61
Episode 20	Last reward: 45.00	Average reward: 17.10
Episode 30	Last reward: 36.00	Average reward: 31.35
Episode 40	Last reward: 104.00	Average reward: 45.32
Episode 50	Last reward: 200.00	Average reward: 62.79
Episode 60	Last reward: 51.00	Average reward: 68.65
Episode 70	Last reward: 30.00	Average reward: 59.67
Episode 80	Last reward: 55.00	Average reward: 54.79
Episode 90	Last reward: 52.00	Average reward: 55.54
Episode 100	Last reward: 62.00	Average reward: 58.38
Episode 110	Last reward: 100.00	Average reward: 69.49
Episode 120	Last reward: 136.00	Average reward: 101.04
Episode 130	Last reward: 162.00	Average reward: 108.17
Episode 140	Last reward: 500.00	Average reward: 204.64
Episode 150	Last reward: 124.00	Average reward: 216.19
Episode 160	Last reward: 125.00	Average reward: 182.25
Episode 170	Last reward: 168.00	Average reward: 168.59
Episode 180	Last reward: 148.00	Average reward: 167.58
Episode 190	Last reward: 248.00	Averag

In [ ]:
plt.plot(numEpisodesList, avgReward)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Reward')
plt.title('Average Reward By Episodes Trained (DQN)')
plt.legend('50 Episodes', '100 Episodes', '150 Episodes', '200 Episodes', '250 Episodes', '300 Episodes', '350 Episodes', '400 Episodes', '450 Episodes', '500 Episodes')
plt.savefig('DQN_Quality_reward.png')

In [ ]:
plt.plot(numEpisodesList, avgTimeToRun)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Time To Run')
plt.title('Average Time to Run By Episodes Trained (DQN)')
plt.savefig('DQN_Quality_time.png')

In [ ]:
print(numEpisodesList)
print(avgReward)

# RUN BELOW FOR EVO

In [ ]:
# Old code assuming random initialization

def init_weights(m):
    if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.00)

def return_random_agents(num_agents):
    agents = []
    for _ in range(num_agents):
        
        agent = CartPoleAI()
        
        for param in agent.parameters():
            param.requires_grad = False
            
        init_weights(agent)
        agents.append(agent)
        
    return agents

In [ ]:
def trainEVOModel_uninit(generationsRun, returnScore = False):
    game_actions = 2
    torch.set_grad_enabled(False)

    agents = return_random_agents(5)

    top_limit = 5 # Number of top agents to consider as parents
    generations = generationsRun

    elite_index = None
    for generation in range(generations):
        rewards = run_agents_n_times(agents, 10) # Average of k runs

        sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit]
        print('\n')

        top_rewards = []
        for best_parent in sorted_parent_indexes:
            top_rewards.append(rewards[best_parent])

        print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
        # print(rewards)
        print("Top ",top_limit," scores", sorted_parent_indexes)
        print("Rewards for top: ",top_rewards)

        children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)
        agents = children_agents
        
    if returnScore == True:
        return np.mean(reward)

In [ ]:
from timeit import default_timer as timer

numEpisodesList = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
numSuccessions = 100

avgReward = np.zeros(len(numEpisodesList))
avgTimeToRun = np.zeros(len(numEpisodesList))

folderName = "EVO_Quality"
fileName   = "Episode"

for ii in range(len(numEpisodesList)):
    
    reward    = 0
    timeToRun = 0

    numEpisodes = numEpisodesList[ii]

    for successions in range(numSuccessions):
        
        print(numEpisodesList[ii])
        print(successions)
        start   = timer()
        reward += trainEVOModel_uninit(numEpisodes, returnScore = True)
        end     = timer()
        
        timeToRun += (end - start)
            
    avgReward[ii]    = reward/numSuccessions
    avgTimeToRun[ii] = timeToRUn/numSuccessions

In [ ]:
plt.plot(numEpisodesList, avgReward)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Reward')
plt.title('Average Reward By Episodes Trained (DQN)')
plt.savefig('EVO_Quality_Reward.png')

In [ ]:
plt.plot(numEpisodesList, avgTimeToRun)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Time To Run')
plt.title('Average Time to Run By Episodes Trained (DQN)')
plt.savefig('EVO_Quality_time.png')

In [ ]:
print(numEpisodesList)
print(avgReward)

# RUN BELOW FOR HYBRID

In [ ]:
from timeit import default_timer as timer

numEpisodesList = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]/2
numSuccessions = 40

avgReward = np.zeros(len(numEpisodesList))
avgTimeToRun = np.zeros(len(numEpisodesList))

folderName = "EVO_Quality"
fileName   = "Episode"

for ii in range(len(numEpisodesList)):
    
    reward    = 0
    timeToRun = 0

    numEpisodes = numEpisodesList[ii]

    for successions in range(numSuccessions):
        
        print(numEpisodesList[ii])
        print(successions)
        start   = timer()
        for jj in range(5):
            policy = Policy()
        optimizer = optim.Adam(policy.parameters(), lr=1.10e-2)
        eps = np.finfo(np.float32).eps.item()
            trainDQNmodel(returnReward = False)
            torch.save(policy.state_dict(), "./" + folderName + "/" + fileName + str(successions))
            
        reward += trainEVOModel(folderName, numEpisodes, returnScore = True)
        
        end     = timer()
        
        timeToRun = (end - start)
            
    avgReward[ii]    = reward/numSuccessions
    avgTimeToRun[ii] = timeToRUn/numSuccessions

In [ ]:
plt.plot(numEpisodesList, avgReward)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Reward')
plt.title('Average Reward By Episodes Trained (DQN)')
plt.savefig('HYBRID_Quality_reward.png')

In [ ]:
plt.plot(numEpisodesList, avgTimeToRun)
plt.xlabel('Num. Ep. Run')
plt.ylabel('Avg. Time To Run')
plt.title('Average Time to Run By Episodes Trained (DQN)')
plt.savefig('HYBRID_Quality_time.png')

In [ ]:
print(numEpisodesList)
print(avgReward)